<a href="https://colab.research.google.com/github/tracirellis/deep_learning_model2/blob/main/starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_cl_df = application_df.drop(["EIN","NAME"],axis=1)
application_cl_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
dataframe_count = application_cl_df.dtypes.index.tolist()
application_df[dataframe_count].nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
df_type_count = application_cl_df['APPLICATION_TYPE'].value_counts()
df_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# use the variable name `application_types_to_replace`
application_types_to_replace = list(df_type_count[df_type_count < 600].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_count = application_cl_df['CLASSIFICATION'].value_counts()
class_count.head(10)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
Name: CLASSIFICATION, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_cl_df['CLASSIFICATION'].value_counts().loc[lambda x : x >1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_count[class_count < 300].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_cl_df)
application_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_dummies['IS_SUCCESSFUL'].values
X = application_dummies.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# installing kera_tuner upgrade
# !pip install keras-tuner --upgrade

# Import the kerastuner library
#import keras_tuner as kt

In [14]:
# 1st try:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 20

nn_model = tf.keras.models.Sequential()

In [15]:
# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                1170      
                                                                 
 dense_1 (Dense)             (None, 20)                220       
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1411 (5.51 KB)
Trainable params: 1411 (5.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:

# Compile the model
nn_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [17]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
804/804 [==============================] - 4s 2ms/step - loss: 0.5883 - accuracy: 0.7067
Epoch 2/25
804/804 [==============================] - 1s 2ms/step - loss: 0.5624 - accuracy: 0.7303
Epoch 3/25
804/804 [==============================] - 1s 2ms/step - loss: 0.5570 - accuracy: 0.7309
Epoch 4/25
804/804 [==============================] - 1s 2ms/step - loss: 0.5540 - accuracy: 0.7324
Epoch 5/25
804/804 [==============================] - 1s 2ms/step - loss: 0.5520 - accuracy: 0.7332
Epoch 6/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7339
Epoch 7/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5487 - accuracy: 0.7346
Epoch 8/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7345
Epoch 9/25
804/804 [==============================] - 1s 2ms/step - loss: 0.5469 - accuracy: 0.7353
Epoch 10/25
804/804 [==============================] - 1s 2ms/step - loss: 0.5460 - accuracy: 0.7344

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5549 - accuracy: 0.7263 - 811ms/epoch - 3ms/step
Loss: 0.5548562407493591, Accuracy: 0.7262973785400391


In [19]:
nn_model.save('AlphabetSoupCharity_Optimization1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
# 2nd try:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 15
hidden_nodes_layer2 = 25
hidden_nodes_layer3 = 25
nn_model = tf.keras.models.Sequential()

In [21]:
# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="sigmoid"))

# 3rd hidden layer
nn_model.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 15)                1755      
                                                                 
 dense_4 (Dense)             (None, 25)                400       
                                                                 
 dense_5 (Dense)             (None, 25)                650       
                                                                 
 dense_6 (Dense)             (None, 1)                 26        
                                                                 
Total params: 2831 (11.06 KB)
Trainable params: 2831 (11.06 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
804/804 [==============================] - 4s 3ms/step - loss: 0.5901 - accuracy: 0.7072
Epoch 2/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5577 - accuracy: 0.7306
Epoch 3/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5534 - accuracy: 0.7324
Epoch 4/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5510 - accuracy: 0.7322
Epoch 5/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7338
Epoch 6/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7330
Epoch 7/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7337
Epoch 8/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5469 - accuracy: 0.7348
Epoch 9/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7347
Epoch 10/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accuracy: 0.7355

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5543 - accuracy: 0.7256 - 697ms/epoch - 3ms/step
Loss: 0.5543205738067627, Accuracy: 0.7255976796150208


In [25]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity_Optimization2.h5')

In [26]:
# 3rd Try:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 30
hidden_nodes_layer2 = 25
hidden_nodes_layer3 = 20
nn_model = tf.keras.models.Sequential()

In [27]:
# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="sigmoid"))

# 3rd hidden layer
nn_model.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 30)                3510      
                                                                 
 dense_8 (Dense)             (None, 25)                775       
                                                                 
 dense_9 (Dense)             (None, 20)                520       
                                                                 
 dense_10 (Dense)            (None, 1)                 21        
                                                                 
Total params: 4826 (18.85 KB)
Trainable params: 4826 (18.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 2s - loss: 0.7363 - accuracy: 0.4668 - 2s/epoch - 6ms/step
Loss: 0.736298143863678, Accuracy: 0.46682214736938477


In [30]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity_Optimization3.h5')